<a href="https://colab.research.google.com/github/surayajohari/STQD6324-Data-Management/blob/main/suraya_assignment2_stqd6324.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [71]:
#Load all data

fd08 = pd.read_csv('/content/drive/MyDrive/STQD6324/flight_data08.csv', sep=',')
airports = pd.read_csv('/content/drive/MyDrive/STQD6324/airports.csv', sep=',')
carriers = pd.read_csv('/content/drive/MyDrive/STQD6324/carriers.csv', sep=',')
plane_data = pd.read_csv('/content/drive/MyDrive/STQD6324/plane-data.csv', sep=',')

In [70]:
fd08['CarrierDelay']

0          16.0
1           NaN
2           NaN
3           NaN
4          16.0
           ... 
2389212     NaN
2389213     NaN
2389214     NaN
2389215     NaN
2389216     NaN
Name: CarrierDelay, Length: 2389217, dtype: float64

In [72]:
# Merge carriers and 2008 data

fd08_carriers_m = pd.merge(fd08, carriers, left_on='UniqueCarrier', right_on='Code', how='left')

In [74]:
# Merge carriers & 2008 data with planes data

fd08_carriers_planes_m = pd.merge(fd08_carriers_m, plane_data, left_on='TailNum', right_on='tailnum', how='left')

In [76]:
# Merge carriers, planes & 2008 data with airport data

# for origin airport
fd08_merged = pd.merge(fd08_carriers_planes_m, airports, left_on='Origin', right_on='iata', how='left')
fd08_merged.rename(columns={'airport':'OriginAirport', 'city':'OriginCity', 'state':'OriginState'}, inplace=True)
fd08_merged.drop(columns=['iata'], inplace=True)

# for destination airport
fd08_merged = pd.merge(fd08_merged, airports, left_on='Dest', right_on='iata', how='left')
fd08_merged.rename(columns={'airport':'DestAirport', 'city':'DestCity', 'state':'DestState'}, inplace=True)
fd08_merged.drop(columns=['iata'], inplace=True)

In [78]:
### 1. What are the optimal times of day, days of the week, and times of the year for minimizing flight delays?

# Convert the date columns to datetime format
fd08_merged['Date'] = pd.to_datetime(fd08_merged['Year'].astype(str) + '-' +
                                    fd08_merged['Month'].astype(str) + '-' +
                                    fd08_merged['DayofMonth'].astype(str))

# Extract day of week (0 = Monday, 6 = Sunday) and hour of the day from the 'Date column
fd08_merged['DayOfWeek']=fd08_merged['Date'].dt.dayofweek
fd08_merged['HourOfDay']=fd08_merged['CRSDepTime'] // 100

# Calculate average delay times grouped by dat of week and hour of day
avg_delay_by_day_hour = fd08_merged.groupby(['DayOfWeek', 'HourOfDay'])['ArrDelay'].mean()

# Find the minimum delay time
min_delay = avg_delay_by_day_hour.min()

# Find the corresponding day of week and hour of day for the minimum delay
optimal_time = avg_delay_by_day_hour.idxmin()
optimal_day_of_week = optimal_time[0]  # index 0 corresponds to DayOfWeek
optimal_hour_of_day = optimal_time[1]  # index 1 corresponds to HourOfDay

# Output
print(f"Optimal time for minimizing flight delays is on {optimal_day_of_week} at {optimal_hour_of_day} o'clock.")
print(f"Average delay time at this optimal time: {min_delay:.2f} minutes.")

Optimal time for minimizing flight delays is on 1 at 4 o'clock.
Average delay time at this optimal time: -8.09 minutes.


In [80]:
### 2. What are the primary factors contributing to flight delays?

# Group by delay factors and sum the delays
delay_factors = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
delay_factors_sum = fd08_merged[delay_factors].sum()

# Identify the primary factor (maximum delay)
primary_delay_factor = delay_factors_sum.idxmax()
max_delay_minutes = delay_factors_sum.max()

# Print results
print(f"The primary factor contributing to flight delays is {primary_delay_factor} with a total delay of {max_delay_minutes} minutes.")

The primary factor contributing to flight delays is LateAircraftDelay with a total delay of 12262628.0 minutes.


In [83]:
### 3. What factors predominantly lead to flight cancellations?

# Count the number of cancellations and group by cancellation reasons
cancellation_reasons = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
cancellation_counts = fd08_merged[cancellation_reasons].sum()

# Identify the predominant cancellation reason (maximum cancellations)
predominant_cancellation_reason = cancellation_counts.idxmax()
max_cancellations = cancellation_counts.max()

# Print results
print(f"The predominant factor leading to flight cancellations is {predominant_cancellation_reason} with {max_cancellations} cancellations.")

The predominant factor leading to flight cancellations is LateAircraftDelay with 12262628.0 cancellations.


In [65]:
### 4. Which flight experiences the most frequent and significant delays and cancellations?

# Group by flight number and calculate counts of delays and cancellations
flight_delays = fd08_merged.groupby('FlightNum')['ArrDelay'].agg(['count', 'mean'])
flight_cancellations = fd08_merged.groupby('FlightNum')['Cancelled'].sum()

# Identify the flight with the most frequent delays (count)
max_delay_count_flight = flight_delays['count'].idxmax()

# Identify the flight with the most significant delays (mean)
max_mean_delay_flight = flight_delays['mean'].idxmax()

# Identify the flight with the most cancellations
max_cancellations_flight = flight_cancellations.idxmax()
max_cancellations_count = flight_cancellations.max()

# Print results
print(f"The flight with the most frequent delays (count) is Flight {max_delay_count_flight}.")
print(f"The flight with the most significant delays (mean) is Flight {max_mean_delay_flight}.")
print(f"The flight with the most cancellations is Flight {max_cancellations_flight} with {max_cancellations_count} cancellations.")


The flight with the most frequent delays (count) is Flight 511.
The flight with the most significant delays (mean) is Flight 7487.
The flight with the most cancellations is Flight 5886 with 70 cancellations.


In [ ]:
### Merging data using Hive

-- Create an external table for data_2008
CREATE EXTERNAL TABLE IF NOT EXISTS data_2008 (
    Year int,
    Month int,
    DayofMonth int,
    DayOfWeek int,
    DepTime string,
    CRSDepTime string,
    ArrTime string,
    CRSArrTime string,
    UniqueCarrier string,
    FlightNum string,
    TailNum string,
    ActualElapsedTime string,
    CRSElapsedTime string,
    AirTime string,
    ArrDelay string,
    DepDelay string,
    Origin string,
    Dest string,
    Distance string,
    TaxiIn string,
    TaxiOut string,
    Cancelled string,
    CancellationCode string,
    Diverted string,
    CarrierDelay string,
    WeatherDelay string,
    NASDelay string,
    SecurityDelay string,
    LateAircraftDelay string
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/path/to/data_2008';

-- Create an external table for carriers
CREATE EXTERNAL TABLE IF NOT EXISTS carriers (
    Code string,
    Description string
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/path/to/carriers';

-- Create an external table for planes
CREATE EXTERNAL TABLE IF NOT EXISTS planes (
    tailnum string,
    type string,
    manufacturer string,
    issue_date string,
    model string,
    status string,
    aircraft_type string,
    engine_type string,
    year int
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/path/to/planes';

-- Create an external table for airports
CREATE EXTERNAL TABLE IF NOT EXISTS airports (
    iata string,
    airport string,
    city string,
    state string,
    country string,
    lat string,
    long string
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/path/to/airports';

-- Merge data_2008 with carriers
CREATE TABLE merged_data AS
SELECT d.*, c.Description as CarrierDescription
FROM data_2008 d
LEFT JOIN carriers c
ON d.UniqueCarrier = c.Code;

-- Merge merged_data with planes
INSERT OVERWRITE TABLE merged_data
SELECT m.*, p.type, p.manufacturer, p.model, p.status
FROM merged_data m
LEFT JOIN planes p
ON m.TailNum = p.tailnum;

-- Merge merged_data with airports for origin
INSERT OVERWRITE TABLE merged_data
SELECT m.*, a.airport as OriginAirport, a.city as OriginCity, a.state as OriginState
FROM merged_data m
LEFT JOIN airports a
ON m.Origin = a.iata;

-- Merge merged_data with airports for destination
INSERT OVERWRITE TABLE merged_data
SELECT m.*, a.airport as DestAirport, a.city as DestCity, a.state as DestState
FROM merged_data m
LEFT JOIN airports a
ON m.Dest = a.iata;

-- Drop intermediate columns if necessary
ALTER TABLE merged_data DROP COLUMN TailNum, UniqueCarrier, Origin, Dest;

-- Show the final merged data
SELECT * FROM merged_data LIMIT 10;
